In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
!pip install tensorflow_decision_forests
!pip install wurlitzer
import tensorflow_decision_forests as tfdf
from wurlitzer import sys_pipes

# Load Data

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')
print('train shape:',train_df.shape)
print('test shape:',test_df.shape)

In [ ]:
feat = train_df.drop(columns = ['id','loss'])
feat.head()

In [ ]:
col_names = list(feat.columns)

# Scale the Data

In [ ]:
scaler = StandardScaler()
train_df[col_names] = scaler.fit_transform(train_df[col_names])
test_df[col_names] = scaler.transform(test_df[col_names])

# Splitting the Train Data

In [ ]:
label = 'loss'
def split_dataset(dataset, test_ratio=0.30):
    """Splits a panda dataframe in two."""
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]

In [ ]:
train_ds_pd, test_ds_pd = split_dataset(train_df.drop(columns=['id']))
# train_ds_pd = train_ds_pd.drop(columns = ['loss'])
# train_ds_pd.head()
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)

# Train the Model

In [ ]:
model = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)
model.compile(metrics=["mse"])

# Train the model.
with sys_pipes():
    model.fit(x=train_ds,validation_data=test_ds)

In [ ]:
import math
evaluation = model.evaluate(test_ds, return_dict=True)

print(evaluation)
print()
print(f"MSE: {evaluation['mse']}")
print(f"RMSE: {math.sqrt(evaluation['mse'])}")

In [ ]:
test_1 = test_df.drop(columns=['id'])

# Prediction using the trained model

In [ ]:
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_1, task=tfdf.keras.Task.REGRESSION)
predictions = model.predict(test_ds)

In [ ]:
predictions.reshape((150000))

In [ ]:
output = pd.DataFrame({'id': test_df.id, 'loss': predictions.reshape((150000))})

In [ ]:
output.to_csv('tfdf_submission.csv', index=False)
print("Your submission was successfully saved!")